In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/proj/')
os.getcwd()

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import math
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split

In [2]:
class BPR(nn.Module):
    def __init__(self):
        super(BPR, self).__init__()
        self.W = None             # user matrix
        self.H = None             # item matrix
        self.Wsc = None           # scorer
        self.Hsc = None 
        
        self.uid = None
        self.iid = None
        
        self.user_items = {}
        self.dev_user_items = {}
        
        self.rating_exp = None   # softmax sum
        self.rating_exp_mul_H = None
    
    def preprocess(self, df, train_size=0.8, test_size=0.1):
        df = df.rename(columns = {df.columns[0]: 'ori_uid', df.columns[1]: 'ori_iid', df.columns[2]: 'rating'})
        df = df.groupby('ori_uid').filter(lambda x: x['ori_uid'].count()>=10)
        uid_map = pd.DataFrame({"ori_uid": np.asarray(list(set(df.iloc[:,0].values)))})
        uid_map["serial_uid"] = uid_map.index
        iid_map = pd.DataFrame({"ori_iid": np.asarray(list(set(df.iloc[:,1].values)))})
        iid_map["serial_iid"] = iid_map.index
        
        self.uid = uid_map["serial_uid"].values
        self.iid = iid_map["serial_iid"].values
        
        df = df.merge(uid_map, left_on = 'ori_uid', right_on = 'ori_uid', how="left")
        df = df.merge(iid_map, left_on = 'ori_iid', right_on = 'ori_iid', how="left")
        df = df[['serial_uid', 'serial_iid', 'rating']]
        
        train, test = self._split(df, train_size)
        test, dev = self._split(test, test_size / (1 - train_size))
        return train, test, dev
    
    def _split(self, df, ratio):
        train = pd.DataFrame(columns = df.columns, dtype=int)
        test = pd.DataFrame(columns = df.columns, dtype=int)
        for i in self.uid:
            train_1, test_1 = train_test_split(df[df.iloc[:, 0] == i], train_size = ratio, shuffle = True, random_state = 5)
            train = pd.concat([train, train_1])
            test = pd.concat([test, test_1])
        return train, test
    
    def generate_train_batch(self, batch, sets):
        train = []
        for b in range(batch):
            u = self.uid[np.random.randint(0, len(self.uid))]
            i = sets[u][np.random.randint(0, len(sets[u]))]
            j = self.iid[np.random.randint(0, len(self.iid))]
            while j in sets[u]:
                j = self.iid[np.random.randint(0, len(self.iid))]
            train.append([u, i, j])
        return np.asarray(train) 

    def forward(self, uids, iids, device):
        self.rating_exp = torch.zeros(len(self.uid)).to(device)
        self.rating_exp_mul_H = torch.zeros([len(self.uid), self.H.shape[1]]).to(device)
        
        # 处理 idx 得到 embedded Wu Hi
        emb_idxs = [self.user_items[uid] for uid in uids]
        item_emb = nn.utils.rnn.pad_sequence([self.Hsc[emb_idx] for emb_idx in emb_idxs], batch_first=True)
        user_emb = self.Wsc[uids][:, None, :]
        
        # 计算批次内 user_item 得分
        user_item_exp_sc = torch.sum(item_emb * user_emb, dim = -1)
        mask = (user_item_exp_sc != 0).type(torch.float32)
        # 取指数， mask 保证补 0 位还是 0
        user_item_exp_sc = torch.exp(user_item_exp_sc) * mask
        
        # 计算指数和
        self.rating_exp_mul_H[uids] = torch.sum(user_item_exp_sc.unsqueeze(2).repeat(1, 1, self.H.shape[1]) * item_emb, dim = 1)
        self.rating_exp[uids] = torch.sum(user_item_exp_sc, dim = 1)
        #返回 softmax probablilty of item i among user_items
        return torch.exp(torch.sum(self.Wsc[uids] * self.Hsc[iids], dim = 1)) / self.rating_exp[uids]
        
    def fit_dds(self, df, dev, k, stepsize=1, max_iter=10, batch=10000, dev_batch=5000, score_stepsize=1):
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.to(device)
        
        # 初始化 W，H
        self.W = nn.Parameter(torch.rand(len(self.uid), k).to(device) * 0.01)   
        self.H = nn.Parameter(torch.rand(len(self.iid), k).to(device) * 0.01) 
        
        # 初始化 scorer
        self.Wsc = torch.rand(len(self.uid), k).to(device) * 0.01    
        self.Hsc = torch.rand(len(self.iid), k).to(device) * 0.01  
        
        # 创建字典：用户u对应他访问过的所有items集合    
        self.user_items = df.groupby(df.columns[0])[df.columns[1]].apply(lambda x: np.array(x)).to_dict()
        self.dev_user_items = dev.groupby(dev.columns[0])[dev.columns[1]].apply(lambda x: np.array(x)).to_dict()
        
        # 主模型优化器        
        optimizer = optim.SGD([self.W, self.H], lr=stepsize)
        
        for x in range(max_iter):            
            #取训练批次：uij三元组
            uij = self.generate_train_batch(batch, self.user_items)
            u = uij[:, 0]
            i = uij[:, 1]
            j = uij[:, 2]
            u_emb = self.W[u]
            i_emb = self.H[i]
            j_emb = self.H[j]
            
            # 评分器概率分布，forward 返回 softmax 概率分布
            score_prob = self.forward(u, i, device)     
            
            # 主模型参数更新
            optimizer.zero_grad() 
            score_loss = -torch.sum(score_prob * torch.log(torch.sigmoid(torch.sum(u_emb * (i_emb-j_emb),dim = 1))))
            bpr_loss = -torch.mean(torch.log(torch.sigmoid(torch.sum(u_emb * (i_emb - j_emb),dim = 1))))
            score_loss.backward()
            optimizer.step()

            # 训练集上 W,H 的梯度
            W_grad_sum = self.W.grad.clone()
            H_grad_sum = self.H.grad.clone()
            
            # 对数概率分布下 Wsc, Hsc 梯度
            log_prob_Wsc_grad = torch.zeros((len(self.uid), k)).to(device)
            log_prob_Hsc_grad = torch.zeros((len(self.iid), k)).to(device)
            log_prob_Wsc_grad[u] = self.Hsc[i] - self.rating_exp_mul_H[u] / self.rating_exp[u].unsqueeze(1).repeat(1, k)
            log_prob_Hsc_grad[i] = self.Wsc[u] * ((1 - score_prob).unsqueeze(1).repeat(1, k))
            
            #取 dev uij三元组
            uij = self.generate_train_batch(dev_batch, self.dev_user_items)
            u = uij[:, 0]
            i = uij[:, 1]
            j = uij[:, 2]
            u_emb = self.W[u]
            i_emb = self.H[i]
            j_emb = self.H[j]
            
            # 计算 dev 集上 W,H 的梯度
            optimizer.zero_grad()
            dev_loss = -torch.sum(torch.log(torch.sigmoid(torch.sum(u_emb * (i_emb - j_emb),dim = 1))))
            dev_loss.backward()
            W_grad_dev_sum = self.W.grad.clone()    
            H_grad_dev_sum = self.H.grad.clone()

            # 计算 reward: reward 为 W,H 在训练集和 dev 集上的梯度积    
            r_W = torch.sum(W_grad_sum * W_grad_dev_sum, dim=1)
            r_H = torch.sum(H_grad_sum * H_grad_dev_sum, dim=1)
            r_W = r_W.unsqueeze(1).repeat(1, k)
            r_H = r_H.unsqueeze(1).repeat(1, k)

            # Wsc，Hsc 更新
            self.Wsc += score_stepsize * r_W * log_prob_Wsc_grad
            self.Hsc += score_stepsize * r_H * log_prob_Hsc_grad

            if ( x + 1 ) % 10 == 0:
                print(f"Iteration: {x+1}, BPR loss: {bpr_loss.item()}")
    
    def fit_ori(self, df, k, stepsize=0.05, max_iter=10, batch=10000):
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.to(device)
        
        self.W = nn.Parameter(torch.rand(len(self.uid), k).to(device) * 0.01)    # 初始化 W，H
        self.H = nn.Parameter(torch.rand(len(self.iid), k).to(device) * 0.01)  
        
        # 创建字典：用户u对应他访问过的所有items集合
        self.user_items = df.groupby(df.columns[0])[df.columns[1]].apply(lambda x: np.array(x)).to_dict()
        
        optimizer = optim.SGD([self.W, self.H], lr=stepsize)     # 主模型优化器
        for x in range(max_iter):
            #取训练批次：uij三元组
            uij = self.generate_train_batch(batch, self.user_items)
            
            u = uij[:, 0]
            i = uij[:, 1]
            j = uij[:, 2]
            u_emb = self.W[u]
            i_emb = self.H[i]
            j_emb = self.H[j]
            
            optimizer.zero_grad()
            loss = -torch.sum(torch.log(torch.sigmoid(torch.sum(u_emb * (i_emb - j_emb),dim = 1))))
            loss.backward()
            optimizer.step()
            
            if ( x + 1 ) % 10 == 0:
                print(f"Iteration: {x+1}, BPR loss: {loss.item() / batch}")
    
    def _predict(self, uid, items, n):
        scores = torch.mv(self.H[items], self.W[uid])
        if n > scores.shape[0]: 
            n = scores.shape[0]
        top_N_val, top_N_idx = torch.topk(scores, k=n)
        return list(zip(items[top_N_idx.cpu()], top_N_val.cpu()))

    def NDCG(self, uid, test, n):         # 用模型排序+真实分数计算 DCG, 重排后计算 iDCG
        test_user = test[test.iloc[:, 0] == uid]
        rating = self._predict(uid, test_user.iloc[:, 1].values, n)
        irating =sorted(test_user.iloc[:, 2].values, reverse=True)
        dcg = 0
        idcg = 0
        if n > len(irating): n = len(irating)  
        for i in range(n):
            r = test_user[test_user.iloc[:, 1]==rating[i][0]].iloc[0, 2]
            dcg += 1.0 * (2**r - 1) / math.log(i + 2, 2)
            idcg += 1.0 * (2**irating[i] - 1) / math.log(i + 2, 2)
        return dcg / idcg

    def performance(self, test, n):      # Output recall@n, precision@n, NDCG@n
        hit = 0
        n_recall = 0
        n_precision = 0
        ndcg = 0
        for i in self.uid:
            # Items that User i hasn't tried in training set
            unknown_items = np.setdiff1d(self.iid, self.user_items[i])
            # Items that User i actually tried in testing set
            known_items = test[test.iloc[:, 0] == i].iloc[:, 1].values
            
            #目标：预测 unknown items 中的top_N，若击中test中的items，则为有效预测
            ru = self._predict(i, unknown_items, n)
            
            hit += sum(1 for item, pui in ru if item in known_items)
            n_recall += len(known_items)
            n_precision += n
            ndcg += self.NDCG(i, test, n)
            
        recall = hit / (1.0 * n_recall)
        precision = hit / (1.0 * n_precision)
        ndcg /= len(self.uid)
        return recall, precision, ndcg

In [2]:
df1 = pd.read_csv("./ml-100k/u.data", sep="\t", names=['user id', 'item id', 'rating', 'timestamp'])
df2 = pd.read_csv("./ml-1m/ratings.dat", sep="::", names=['user id', 'item id', 'rating', 'timestamp'], engine='python')

### 100K

In [4]:
model1 = BPR()
model2 = BPR()
train1, test1, dev1 = model1.preprocess(df1)
train2, test2, dev2 = model2.preprocess(df1)
print(train1.shape)
print(test1.shape)
print(dev1.shape)

(79619, 3)
(9942, 3)
(10439, 3)


### 100K Pure BPR

In [23]:
%%time
model1.fit_ori(train1, k = 50, max_iter = 500)

Iteration: 10, BPR loss: 0.62280771484375
Iteration: 20, BPR loss: 0.354079931640625
Iteration: 30, BPR loss: 0.318456787109375
Iteration: 40, BPR loss: 0.3061577392578125
Iteration: 50, BPR loss: 0.2965654052734375
Iteration: 60, BPR loss: 0.2616852294921875
Iteration: 70, BPR loss: 0.2472632080078125
Iteration: 80, BPR loss: 0.219330615234375
Iteration: 90, BPR loss: 0.192760595703125
Iteration: 100, BPR loss: 0.1887091552734375
Iteration: 110, BPR loss: 0.17070421142578124
Iteration: 120, BPR loss: 0.16379403076171875
Iteration: 130, BPR loss: 0.154899951171875
Iteration: 140, BPR loss: 0.149299267578125
Iteration: 150, BPR loss: 0.1481927734375
Iteration: 160, BPR loss: 0.14233681640625
Iteration: 170, BPR loss: 0.1392462158203125
Iteration: 180, BPR loss: 0.1381890625
Iteration: 190, BPR loss: 0.1325078369140625
Iteration: 200, BPR loss: 0.1253777099609375
Iteration: 210, BPR loss: 0.1216952880859375
Iteration: 220, BPR loss: 0.1112137939453125
Iteration: 230, BPR loss: 0.10821766

In [24]:
%%time
n = 10
rec, pre, ndcg = model1.performance(test1, n)
print(f'Precision@{n}: {pre}')
print(f'Recall@{n}: {rec}')
print(f'NDCG@{n}: {ndcg}')

Precision@10: 0.1366914103923648
Recall@10: 0.12965198149265741
NDCG@10: 0.8255023160063666
CPU times: total: 2.38 s
Wall time: 3.11 s


### 100K BPR + Data Selection

In [5]:
%%time
model2.fit_dds(train1, dev1, k = 50, max_iter = 500)

Iteration: 10, BPR loss: 0.6657573580741882
Iteration: 20, BPR loss: 0.49578118324279785
Iteration: 30, BPR loss: 0.42297127842903137
Iteration: 40, BPR loss: 0.37862229347229004
Iteration: 50, BPR loss: 0.3530023694038391
Iteration: 60, BPR loss: 0.3378726541996002
Iteration: 70, BPR loss: 0.3367163836956024
Iteration: 80, BPR loss: 0.31973254680633545
Iteration: 90, BPR loss: 0.31463074684143066
Iteration: 100, BPR loss: 0.30432575941085815
Iteration: 110, BPR loss: 0.2773972749710083
Iteration: 120, BPR loss: 0.2752606272697449
Iteration: 130, BPR loss: 0.25736239552497864
Iteration: 140, BPR loss: 0.2483510673046112
Iteration: 150, BPR loss: 0.230539932847023
Iteration: 160, BPR loss: 0.21782223880290985
Iteration: 170, BPR loss: 0.21002554893493652
Iteration: 180, BPR loss: 0.21014492213726044
Iteration: 190, BPR loss: 0.1985919177532196
Iteration: 200, BPR loss: 0.19996725022792816
Iteration: 210, BPR loss: 0.18652160465717316
Iteration: 220, BPR loss: 0.1823459267616272
Iteratio

In [14]:
%%time
n = 10
rec, pre, ndcg = model2.performance(test1, n)
print(f'Precision@{n}: {pre}')
print(f'Recall@{n}: {rec}')
print(f'NDCG@{n}: {ndcg}')

Precision@10: 0.15874867444326618
Recall@10: 0.15057332528666265
NDCG@10: 0.8317444912367737
CPU times: total: 1.92 s
Wall time: 3.17 s


### 1M

In [6]:
model3 = BPR()
model4 = BPR()
train3, test3, dev3 = model3.preprocess(df2)
train4, test4, dev4 = model4.preprocess(df2)
print(train3.shape)
print(test3.shape)
print(dev3.shape)

(797758, 3)
(99692, 3)
(102759, 3)


### 1M Pure BPR

In [6]:
%%time
model3.fit_ori(train3, k = 20, max_iter = 500, stepsize=0.1)

Iteration: 10, BPR loss: 0.68849326171875
Iteration: 20, BPR loss: 0.5787048828125
Iteration: 30, BPR loss: 0.4073212890625
Iteration: 40, BPR loss: 0.351703271484375
Iteration: 50, BPR loss: 0.343032470703125
Iteration: 60, BPR loss: 0.3304013671875
Iteration: 70, BPR loss: 0.331921044921875
Iteration: 80, BPR loss: 0.3256033203125
Iteration: 90, BPR loss: 0.330059521484375
Iteration: 100, BPR loss: 0.3271123046875
Iteration: 110, BPR loss: 0.315700634765625
Iteration: 120, BPR loss: 0.321971240234375
Iteration: 130, BPR loss: 0.3211672119140625
Iteration: 140, BPR loss: 0.31386025390625
Iteration: 150, BPR loss: 0.311345263671875
Iteration: 160, BPR loss: 0.31525048828125
Iteration: 170, BPR loss: 0.3073806884765625
Iteration: 180, BPR loss: 0.2932947265625
Iteration: 190, BPR loss: 0.281919189453125
Iteration: 200, BPR loss: 0.2668001708984375
Iteration: 210, BPR loss: 0.245215478515625
Iteration: 220, BPR loss: 0.2367917724609375
Iteration: 230, BPR loss: 0.2350786376953125
Iterati

In [7]:
%%time
n = 10
rec, pre, ndcg = model3.performance(test3, n)
print(f'Precision@{n}: {pre}')
print(f'Recall@{n}: {rec}')
print(f'NDCG@{n}: {ndcg}')

Precision@10: 0.13413907284768212
Recall@10: 0.0812703125626931
NDCG@10: 0.8072148664429862
CPU times: total: 18.1 s
Wall time: 27 s


### 1M BPR + Data Selection

In [7]:
%%time
model4.fit_dds(train3, dev3, k=20, max_iter=500, stepsize=4, score_stepsize=1.5)

Iteration: 10, BPR loss: 0.6872056722640991
Iteration: 20, BPR loss: 0.5791022181510925
Iteration: 30, BPR loss: 0.466282457113266
Iteration: 40, BPR loss: 0.41698288917541504
Iteration: 50, BPR loss: 0.3944404125213623
Iteration: 60, BPR loss: 0.37079060077667236
Iteration: 70, BPR loss: 0.3681659400463104
Iteration: 80, BPR loss: 0.3572051525115967
Iteration: 90, BPR loss: 0.3441734313964844
Iteration: 100, BPR loss: 0.35157909989356995
Iteration: 110, BPR loss: 0.3355865776538849
Iteration: 120, BPR loss: 0.32879626750946045
Iteration: 130, BPR loss: 0.33184167742729187
Iteration: 140, BPR loss: 0.31641504168510437
Iteration: 150, BPR loss: 0.307462602853775
Iteration: 160, BPR loss: 0.2963956594467163
Iteration: 170, BPR loss: 0.29591602087020874
Iteration: 180, BPR loss: 0.29209789633750916
Iteration: 190, BPR loss: 0.27236083149909973
Iteration: 200, BPR loss: 0.2658662796020508
Iteration: 210, BPR loss: 0.26851415634155273
Iteration: 220, BPR loss: 0.25188401341438293
Iteration:

In [8]:
%%time
model4.fit_dds(train3, dev3, k=20, max_iter=500, stepsize=4)

Iteration: 10, BPR loss: 0.6871827244758606
Iteration: 20, BPR loss: 0.5806105732917786
Iteration: 30, BPR loss: 0.4621551036834717
Iteration: 40, BPR loss: 0.41256240010261536
Iteration: 50, BPR loss: 0.3887789249420166
Iteration: 60, BPR loss: 0.37127685546875
Iteration: 70, BPR loss: 0.35713833570480347
Iteration: 80, BPR loss: 0.3605228066444397
Iteration: 90, BPR loss: 0.35070475935935974
Iteration: 100, BPR loss: 0.3333551585674286
Iteration: 110, BPR loss: 0.33875972032546997
Iteration: 120, BPR loss: 0.32622358202934265
Iteration: 130, BPR loss: 0.3262203633785248
Iteration: 140, BPR loss: 0.31632691621780396
Iteration: 150, BPR loss: 0.32030531764030457
Iteration: 160, BPR loss: 0.3096601366996765
Iteration: 170, BPR loss: 0.299712598323822
Iteration: 180, BPR loss: 0.29073184728622437
Iteration: 190, BPR loss: 0.27969589829444885
Iteration: 200, BPR loss: 0.2653302550315857
Iteration: 210, BPR loss: 0.2544574737548828
Iteration: 220, BPR loss: 0.25236785411834717
Iteration: 2

In [8]:
%%time
n = 10
rec, pre, ndcg = model4.performance(test3, n)
print(f'Precision@{n}: {pre}')
print(f'Recall@{n}: {rec}')
print(f'NDCG@{n}: {ndcg}')

Precision@10: 0.12145695364238411
Recall@10: 0.07358664687236689
NDCG@10: 0.8168880722578074
CPU times: total: 17.8 s
Wall time: 25.2 s


In [ ]:
def forward(self, uids, iids, device):
        self.rating_exp = torch.zeros(len(self.uid)).to(device)
        self.rating_exp_mul_H = torch.zeros([len(self.uid), self.H.shape[1]]).to(device)

        # 处理 idx 得到 embedded Wu Hi
        emb_idxs = [self.user_items[uid] for uid in uids]
        item_emb = nn.utils.rnn.pad_sequence([self.Hsc[emb_idx] for emb_idx in emb_idxs], batch_first=True)
        user_emb = self.Wsc[uids][:, None, :]

        # 计算批次内 user_item 得分
        user_item_exp_sc = torch.sum(item_emb * user_emb, dim = -1)
        max_sc_per_row = torch.max(user_item_exp_sc, 1).values

        mask = (user_item_exp_sc != 0).type(torch.float32)
        user_item_exp_sc = user_item_exp_sc - max_sc_per_row[:, None]

        # 取指数， mask 保证补 0 位还是 0
        user_item_exp_sc = torch.exp(user_item_exp_sc) * mask

        # 计算指数和
        self.rating_exp_mul_H[uids] = torch.sum(user_item_exp_sc.unsqueeze(2).repeat(1, 1, self.H.shape[1]) * item_emb, dim = 1)
        self.rating_exp[uids] = torch.sum(user_item_exp_sc, dim = 1)

        #返回 softmax probablilty of item i among user_items
        return torch.exp(torch.sum(self.Wsc[uids] * self.Hsc[iids], dim = 1) - max_sc_per_row) / self.rating_exp[uids]